We have borrowed the WEAT word sets from two different works. Some word sets occur more times with slightly different words. In orderr to make the test more uniform, we build other WEAT word sets by taking the union of the ones found in the two reference worrks.

In addition, we add the most frequent male and female proper names of song lyrics.

In [1]:
import json
import pandas as pd

In [2]:
# load original WEAT word sets
weat1 = json.load(open("../data/Data_WEAT/weat_attrib_target.json"))
weat2 = json.load(open("../data/Data_WEAT/weat_attrib_target_2.json"))

# load new properr names
new_male_names = open("../data/Data_WEAT/new_male_names.txt", 'rt').read().split("\n")
new_male_names = [n.lower() for n in new_male_names if n!='']

new_female_names = open("../data/Data_WEAT/new_female_names.txt", 'rt').read().split("\n")
new_female_names = [n.lower() for n in new_female_names if n!='']

In [3]:
# collect all words in each word set
all_word_sets = []
tests = []

for k, val in weat1.items():
    if k.startswith("_") or val['method']=='wefat': continue
                
    A_key = val['A_key']
    B_key = val['B_key']
    X_key = val['X_key']
    Y_key = val['Y_key']
    
    A_words = val[A_key]
    B_words = val[B_key]
    X_words = val[X_key]
    Y_words = val[Y_key]
    
    all_word_sets.append(['weat1', 'attribute', A_key, A_words])
    all_word_sets.append(['weat1', 'attribute', B_key, B_words])
    all_word_sets.append(['weat1', 'target', X_key, X_words])
    all_word_sets.append(['weat1', 'target', Y_key, Y_words])
    
    tests.append(['weat1', k, A_key, B_key, X_key, Y_key])
    
for k, val in weat2.items():
    if k.startswith("_") or val['method']=='wefat': continue
        
    A_key = val['A_key']
    B_key = val['B_key']
    X_key = val['X_key']
    Y_key = val['Y_key']
    
    A_words = val[A_key]
    B_words = val[B_key]
    X_words = val[X_key]
    Y_words = val[Y_key]
    
    all_word_sets.append(['weat2', 'attribute', A_key, A_words])
    all_word_sets.append(['weat2', 'attribute', B_key, B_words])
    all_word_sets.append(['weat2', 'target', X_key, X_words])
    all_word_sets.append(['weat2', 'target', Y_key, Y_words])
    
    tests.append(['weat2', k, A_key, B_key, X_key, Y_key])

# the WEAT1 has two tests with same target and attribute names
tests = pd.DataFrame(tests, columns=['weat', 'test_name', 'A', 'B', 'X', 'Y'])
tests = tests.drop_duplicates().reset_index(drop=True)

all_word_sets = pd.DataFrame(all_word_sets, columns=['weat', 'set_kind', 'set_name', 'words'])
all_word_sets.head()

,weat,set_kind,set_name,words
0,weat1,attribute,Pleasant,"[caress, freedom, health, love, peace, cheer, ..."
1,weat1,attribute,Unpleasant,"[abuse, crash, filth, murder, sickness, accide..."
2,weat1,target,European American names,"[adam, chip, harry, josh, roger, alan, frank, ..."
3,weat1,target,African American names,"[alonzo, jamel, lerone, percell, theo, alphons..."
4,weat1,attribute,Pleasant,"[caress, freedom, health, love, peace, cheer, ..."


In [4]:
# take the union of the names of each set with same name
flatten_list = lambda ll: [i for l in ll for i in l] 

all_word_sets_uniq = all_word_sets.groupby("set_name")['words'].apply(lambda words: 
                                                             list(set(flatten_list(words.tolist()))))
all_word_sets_uniq = all_word_sets_uniq.reset_index()
all_word_sets_uniq.sort_index()

,set_name,words
0,African American names,"[aisha, tyrone, teretha, jasmine, tashika, tia..."
1,Appearance words,"[slim, thin, attractive, feeble, healthy, stro..."
2,Arts words,"[poetry, novel, symphony, art, dance, shakespe..."
3,Career words,"[corporation, professional, executive, busines..."
4,European American names,"[peggy, shannon, allison, brad, crystal, jonat..."
5,Family words,"[relatives, family, marriage, wedding, parents..."
6,Female attributes,"[girl, hers, her, aunt, daughter, sister, fema..."
7,Female names,"[diana, kate, joan, sarah, donna, lisa, amy, ann]"
8,Flowers,"[lilac, bluebell, violet, zinnia, peony, crocu..."
9,Insects,"[cockroach, wasp, bedbug, ant, beetle, gnat, f..."


In [5]:
# create new sets:

all_word_sets_uniq = all_word_sets_uniq.set_index('set_name')

# science words + math words -> matsci words
all_word_sets_uniq.loc['MatSci words', 'words'] = all_word_sets_uniq.loc['Science words'].words + \
                                                    all_word_sets_uniq.loc['Math words'].words

# Male names new and Female names new
all_word_sets_uniq.loc['Male names new', 'words'] = new_male_names
all_word_sets_uniq.loc['Female names new', 'words'] = new_female_names

all_word_sets_uniq

,words
set_name,
African American names,"[aisha, tyrone, teretha, jasmine, tashika, tia..."
Appearance words,"[slim, thin, attractive, feeble, healthy, stro..."
Arts words,"[poetry, novel, symphony, art, dance, shakespe..."
Career words,"[corporation, professional, executive, busines..."
European American names,"[peggy, shannon, allison, brad, crystal, jonat..."
Family words,"[relatives, family, marriage, wedding, parents..."
Female attributes,"[girl, hers, her, aunt, daughter, sister, fema..."
Female names,"[diana, kate, joan, sarah, donna, lisa, amy, ann]"
Flowers,"[lilac, bluebell, violet, zinnia, peony, crocu..."


### Make new tests

In [6]:
tests.head()

,weat,test_name,A,B,X,Y
0,weat1,EuropeanAmerican_AfricanAmerican_Pleasant_Unpl...,Pleasant,Unpleasant,European American names,African American names
1,weat1,EuropeanAmerican_AfricanAmerican_Pleasant_Unpl...,Pleasant,Unpleasant,European American names,African American names
2,weat1,Flowers_Insects_Pleasant_Unpleasant,Pleasant,Unpleasant,Flowers,Insects
3,weat1,Male_Female_Career_Family,Career words,Family words,Male names,Female names
4,weat1,Math_Arts_Male_Female,Male attributes,Female attributes,Math words,Arts words


In [7]:
# remove test with African American names because of few occurrences
# then take unique tests
tests = tests[tests.Y!='African American names']
tests = tests.drop_duplicates(subset=['A', 'B', 'X', 'Y'])
tests = tests.drop(columns=['weat'])

# add new tests
new_tests = [
    {'test_name': 'Matsci_Arts_Male_Female', 'A':'Male attributes', 'B':'Female attributes', 
     'X':'MatSci words', 'Y':'Arts words'},
    {'test_name': 'Malenew_Femalenew_Career_Family', 'A':'Career words', 'B':'Family words', 
     'X':'Male names new', 'Y':'Female names new'},
]

tests = tests.append(new_tests).reset_index(drop=True)
tests.head()

,test_name,A,B,X,Y
0,Flowers_Insects_Pleasant_Unpleasant,Pleasant,Unpleasant,Flowers,Insects
1,Male_Female_Career_Family,Career words,Family words,Male names,Female names
2,Math_Arts_Male_Female,Male attributes,Female attributes,Math words,Arts words
3,MusicalInstruments_Weapons_Pleasant_Unpleasant,Pleasant,Unpleasant,Musical instruments,Weapons
4,Science_Arts_Male_Female,Male attributes,Female attributes,Science words,Arts words


In [8]:
weat3 = {}

for idx, row in tests.iterrows():
    
    test_name = row.test_name
    A_key = row.A
    B_key = row.B
    X_key = row.X
    Y_key = row.Y
    
    A_words = all_word_sets_uniq.loc[A_key].words
    B_words = all_word_sets_uniq.loc[B_key].words
    X_words = all_word_sets_uniq.loc[X_key].words
    Y_words = all_word_sets_uniq.loc[Y_key].words
    
    new_test = {
        'A_key' : A_key,
        A_key : A_words,
        'B_key' : B_key,
        B_key : B_words,
        'X_key' : X_key,
        X_key : X_words,
        'Y_key' : Y_key,
        Y_key : Y_words,
        'attributes' : A_key + ' vs ' + B_key,
        'targets' : X_key + ' vs ' + Y_key,
        'method':'weat'
    }
    weat3[test_name] = new_test

In [9]:
json.dump(weat3, open("../data/Data_WEAT/weat_attrib_target_3.json", 'wt'))